<h2>Netflix Movies and TV Shows</h2>


*   This dataset consists of tv shows and movies available on Netflix
as of 2019.
*   The dataset is collected from Flexible which is a third-party Netflix search engine.
*   In 2018, they released an interesting report which shows that the number of TV shows on Netflix
has nearly tripled since 2010.
*   The streaming service’s number of movies has decreased by
more than 2,000 titles since 2010, while its number of TV shows has nearly tripled

*Aim:   Build a model to predictrating for movies and TV shows*





<h3> Impoting the basic pakages and the dataset </h3>


In [268]:
# Importing some basic pkg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [269]:
# Reading the csv file (upload the dataset and import)
# Link: https://drive.google.com/file/d/1sHBjusvlYDQEhOAweqZvCr3NUeqp5P8O/view?usp=sharing
df = pd.read_csv("/content/netflix_titles.csv")

1. Analysis of the data

In [270]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [271]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [272]:
print(df["rating"].value_counts())
print("\n",df.iloc[5397,:])
# so we can remove the last entry as its completely na

TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
66 min         1
74 min         1
84 min         1
Name: rating, dtype: int64

 show_id                                                     s5398
type                                                      TV Show
title           Learning Songs by Little Baby Bum: Nursery Rhy...
director                                                      NaN
cast                    Chloe Marsden, Aaron Marsden, María Esnoz
country                                            United Kingdom
date_added                                           July 1, 2017
release_year                                                 2015
rating                                                       TV-Y
duration                                                 1 Season
listed_in             

2. Taking inputs and outputs
*    Input / features:  ( concatenated together)
              "Title"
              "listed_in"
              "description" 
*    Output / Target: 
              "rating"

In [273]:
# Inputs and outputs, excluding the last one which is NaN
X = df.iloc[:5397,[2,10,11]].values 
y = df.iloc[:5397,8].values

3. Some imports for the model proceeded by lemmatizing and stemming

*    Stemmer: 

```
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
```
*    Lemmetizer:

```
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
```





In [274]:
# importing from sklearn
from sklearn.model_selection import train_test_split as tts
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

In [275]:
# importing nltk
import nltk 
nltk.download('stopwords') #stopwords download
nltk.download('punkt') #tokenizer download
nltk.download('wordnet') #lemmatizer download

# stemming and lemmatization imports
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
new_stopword=stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [276]:
# stemming and lemmatization
X_new = []
for _ in X:
  temp = []
  for i in _:
    review = re.sub('[^a-zA-Z]',' ',str(i))
    review = review.lower()
    review = review.split()
    new_review = [lemmatizer.lemmatize(stemmer.stem(word)) for word in review if not word in new_stopword]
    new_review = ' '.join(new_review) 
    temp.append(new_review)
  X_new.append(' '.join(temp))

4. Pipeling and and fitting the model
*    Tokenizer used: ```CountVectorizer()```
*    Model used: ```LinearSVC```  

In [277]:
# splitting dataset, we are only using the test data 
X_train, X_test, y_train, y_test = tts(X_new,y,test_size=.75, random_state = 2)

In [278]:
# pipeling and fitting with all data
param_grid = {'C': [0.01, 0.1, 1.0, 10.0, 100.0]}
model = make_pipeline(CountVectorizer(), GridSearchCV(LinearSVC(loss='hinge',random_state=2), param_grid))
model.fit(X_new, y)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('gridsearchcv',
                 GridSearchCV(estimator=LinearSVC(loss='hinge', random_state=2),
                              param_grid={'C': [0.01, 0.1, 1.0, 10.0,
                                                100.0]}))])

5. Testing and checking the accuracy

In [279]:
y_pred = model.predict(X_test)

In [280]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_pred, y_test)

0.9239130434782609

In [281]:
print(classification_report(y_pred, y_test,zero_division=0))

              precision    recall  f1-score   support

           G       1.00      1.00      1.00        10
       NC-17       0.00      0.00      0.00         0
          PG       0.99      0.97      0.98        94
       PG-13       0.92      0.97      0.95       151
           R       0.94      0.97      0.95       265
       TV-14       0.85      0.93      0.89       956
        TV-G       0.95      0.99      0.97       107
       TV-MA       0.96      0.89      0.92      1818
       TV-PG       0.87      0.99      0.93       311
        TV-Y       0.96      0.98      0.97       182
       TV-Y7       1.00      0.95      0.98       154

    accuracy                           0.92      4048
   macro avg       0.86      0.88      0.87      4048
weighted avg       0.93      0.92      0.92      4048

